# Install Packages

In [2]:
# ! pip install python-dotenv
# ! pip -q install --upgrade langchain-openai python-docx PyPDF2 pypdf
# ! pip install chromadb langchain openai langchain-chroma chroma-migrate

In [3]:
import chromadb
print("ChromaDB SDK version:", chromadb.__version__)

ChromaDB SDK version: 0.5.23


# Load Secrets

In [5]:
import os
from dotenv import load_dotenv
from pathlib import Path

# Move up one directory to find .env
env_path = Path.cwd().parent / '.env'

# Load the .env file
load_dotenv(dotenv_path=env_path)

# Access your keys
groq_api_key = os.getenv('GROQ_API_KEY')
openai_api_key = os.getenv('OPENAI_API_KEY')

print(f"GROQ_API_KEY: {bool(groq_api_key)}")
print(f"OPENAI_API_KEY: {bool(openai_api_key)}")


GROQ_API_KEY: True
OPENAI_API_KEY: True


# Initialize ChromaDB & LangChain

In [7]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [8]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_db",  # Where to save data locally, remove if not necessary
)


print("✅ LangChain + Chroma vector store ready (persistent, local)!")

✅ LangChain + Chroma vector store ready (persistent, local)!


# Load your files, split into chunks, and upsert into ChromaDB

In [10]:
import os
from langchain.document_loaders import PyPDFLoader, Docx2txtLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

DATA_DIR = "./data"

# 1. Load & tag raw docs
raw_docs = []
for fname in os.listdir(DATA_DIR):
    path = os.path.join(DATA_DIR, fname)
    if fname.lower().endswith(".pdf"):
        loader = PyPDFLoader(path)
    elif fname.lower().endswith(".docx"):
        loader = Docx2txtLoader(path)
    elif fname.lower().endswith(".txt"):
        loader = TextLoader(path, encoding="utf-8")
    else:
        continue

    docs = loader.load()
    # attach filename so we can delete/update later
    for d in docs:
        d.metadata["source"] = fname
    raw_docs.extend(docs)

print(f"Loaded {len(raw_docs)} document chunks from {len(os.listdir(DATA_DIR))} files.")

# 2. Split into manageable pieces
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = splitter.split_documents(raw_docs)
print(f"Split into {len(chunks)} total chunks.")

Loaded 53 document chunks from 2 files.
Split into 179 total chunks.


In [11]:
print(chunks[0])

page_content='Addendum: Fitting the DESI BAO Data with Dark
Energy Driven by the Cohen–Kaplan–Nelson Bound
Patrick Adolf1∗, Martin Hirsch2†, Sara Krieg1‡, Heinrich P¨ as1§, Mustafa Tabet1¶
1Fakult¨ at f¨ ur Physik, Technische Universit¨ at Dortmund, D-44221 Dortmund, Germany
2Instituto de F` ısica Corpuscular (IFIC), Universidad de Valencia-CSIC,
E-46980 Valencia, Spain
April 23, 2025
Abstract
Motivated by the recent Year-2 data release of the DESI collaboration, we update our results on time-
varying dark energy models driven by the Cohen–Kaplan–Nelson bound. The previously found preference of
time-dependent dark energy models compared to ΛCDM is further strengthend by the new data release. For
our particular models, we find that this preference increases up to ≈ 2.6 σ depending on the used supernova
dataset.
1 Introduction
In this addendum, we update the results of our previous work [1] in the light of the recent Year-2 data release' metadata={'producer': 'pdfTeX-1.40.25', 'creator':

In [12]:
# For a single document
print(chunks[0].metadata['source'])

# # To print the file names of all documents
# for doc in chunks:
#     print(doc.metadata['source'])

astro_physics_1.pdf


# Insert Embeddings into ChromaDB

In [14]:
# 3. Upsert your document chunks
vector_store.add_documents(chunks)

# 4. Write everything immediately to disk
# vector_store.persist()

print(f"✔️ Ingested {len(chunks)} chunks into 'file_embeddings' collection.")

✔️ Ingested 179 chunks into 'file_embeddings' collection.


# Filter and Query Using Filename

In [16]:
# # Replace with your actual filename
file_name = "CSE_1.pdf"

col = vector_store._collection

#  Use a metadata “where” filter to get only that file’s vectors
results = col.get(
    where={"source": file_name}
    # include=["metadatas"]    # <-- you can include metadatas/documents if you want, but not "ids"
)
vector_ids = results["ids"]

print(f"🔍 Found {len(vector_ids)} embeddings for '{file_name}':\n{vector_ids}")

🔍 Found 978 embeddings for 'CSE_1.pdf':
['72c7c99c-2353-4cb0-b062-9d164da1811d', '85965409-3ee3-4e67-aa33-837b6987b3ca', '10b10807-24cb-4c24-adba-8043ece05a44', '4d9146e9-d15e-4f13-98ff-68c2d70e9fe4', 'da4cec38-1d39-48dd-a0b5-9c7794d8bff2', '3d1a1341-d95f-42a3-974e-4dfe5e89157d', '9fe62d75-36fa-4f37-b6f8-d93bf1717c4a', '34a38db5-e823-4dbc-aea9-00a7b316f2a5', 'cb40398f-5a0f-4a06-88fb-31d89ef3013a', 'd7626970-b48c-49f4-8f78-010bb9c79a57', 'dc2aac5e-bbf3-4d86-a343-5056327d3b98', '5e42eed3-9fef-490a-9b6c-ee3dc34273eb', '7349472c-13aa-4098-8dcd-10a62adc792f', '69633824-8d2a-4941-bb8f-b05c66585af9', 'a9ef68ea-213a-4e01-86ad-1776e3565d57', '310b471d-1481-454b-9beb-43200549e07d', 'b4d67271-386d-4ef2-8d33-6ebc02598b39', '132a34d0-80c5-41d1-a3f7-e7b5aab5dfd3', '483a14a0-4da7-46cc-8dad-7cb92b0136c9', 'a0feb1b1-496b-45af-93a2-124aafce6b0b', 'b4f2bb00-5554-409d-bfc8-08d3842f567e', '81d8b252-6642-4cc0-9bcf-a6f0fabd5854', '423406f6-d11c-4e56-ac88-0b561138cbcb', '464f7d80-146e-4a2c-8cb0-c33c98742f3b',

# Create a Retriever (For RAG)

In [18]:
# Step 1: build your retriever (no extra imports needed)
retriever =  vector_store.as_retriever(
    search_type="mmr", search_kwargs={"k": 1, "fetch_k": 5}
)
print("Retriever initialized:", retriever)

Retriever initialized: tags=['Chroma', 'OpenAIEmbeddings'] vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x0000012AB0634B30> search_type='mmr' search_kwargs={'k': 1, 'fetch_k': 5}


# Create the RetrievalQA chain

In [20]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

# 1. Instantiate an LLM
llm = ChatOpenAI(temperature=0)

# 2. Build the RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

print("✅ RetrievalQA chain is ready")


✅ RetrievalQA chain is ready


# Test your RetrievalQA chain

In [22]:
# 1. Define your query
# query = "What are the main topics covered in CSE_1.pdf?"
query = "Tell me about Learning in Dynamic Bayesian Stackelberg Games"

# 2. Run the RetrievalQA chain
result =  qa_chain.invoke({"query": query})

# 3. Print the answer
print("📝 Answer:\n", result["result"])

# 4. Inspect source documents
print("\n🔍 Source chunks:")
for doc in result["source_documents"]:
    print(f"- {doc.metadata['source']} (chunk snippet: {doc.page_content[:150]}...)")

📝 Answer:
 In Learning in Dynamic Bayesian Stackelberg Games, the leader's focus is on maximizing their utility over the rounds of the game rather than identifying the follower's type, which is common in traditional learning paradigms. The leader plays a Dynamic Bayesian Stackelberg Policy (DSP) that specifies their strategy at each round, committing to this policy before the game starts. The follower observes this policy in advance. The optimal DSP is known as the Dynamic Bayesian Stackelberg Equilibrium (DSE). This approach is used in various fields such as dynamic pricing problems, dynamic mechanism design, and Stackelberg security games.

🔍 Source chunks:
- CSE_1.pdf (chunk snippet: drawn from µ. We denote a specific dynamic Bayesian Stackelberg game as {R, Θ, {Cθ}θ∈Θ, µ, T}.
The leader plays a Dynamic Bayesian Stackelberg Policy ...)


# Initiate Agno Agent

In [24]:
from agno.agent import Agent
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.knowledge.langchain import LangChainKnowledgeBase
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from pydantic import BaseModel, Field
from typing import List

In [25]:
# Knowledge Base
knowledge_base = LangChainKnowledgeBase(retriever=retriever)


In [26]:
# Define Pydantic model for structured output
class AnswerOutput(BaseModel):
    query: str = Field(..., description="The user's original query.")
    answer: str = Field(..., description="The answer to the user's query.")
    type: str = Field(..., description="Source of the answer: 'Knowledge/RAG' or 'Web Search'.")
    sources: List[str] = Field(..., description="List of sources used (URLs or document names).")


In [27]:
# Agent with structured_output using Pydantic model
agent = Agent(
    tools=[DuckDuckGoTools()],
    knowledge=knowledge_base,
    instructions=[
        "Try to answer the query using the knowledge base (RAG) first.",
        "If the knowledge base does not contain the information or the user asks for a web search, use DuckDuckGo.",
        "Return your response following the provided structured format."
    ],
    response_model=AnswerOutput,  # <-- Pydantic model here
    show_tool_calls=True,
    markdown=False,
)

# Test the agent

In [29]:
from agno.agent import RunResponse
from rich.pretty import pprint
from datetime import datetime

start_time = datetime.now()
print(f"Start Time: {start_time.strftime('%Y-%m-%d %H:%M:%S')}")

# Test the agent
# response: RunResponse = agent.run("What are the latest developments in AI, and how do they relate to the concepts in our knowledge base?")
response: RunResponse = agent.run("Tell me about Learning in Dynamic Bayesian Stackelberg Games")

end_time = datetime.now()
elapsed = end_time - start_time

print(f"End Time: {end_time.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Elapsed Time: {elapsed.total_seconds():.2f} seconds")  # <-- FIXED


Start Time: 2025-06-08 15:15:10
End Time: 2025-06-08 15:15:18
Elapsed Time: 8.41 seconds


In [30]:
# Pretty print the structured content (your AnswerOutput model)
pprint(response.content)

AnswerOutput(
│   query='Learning in Dynamic Bayesian Stackelberg Games',
│   answer="In Dynamic Bayesian Stackelberg Games, the focus is on how the leader can optimize their utility over multiple rounds of the game rather than merely identifying the follower's type. The leader employs a Dynamic Bayesian Stackelberg Policy (DSP), which specifies the strategy at each round based on the follower's past responses. This policy outlines a leader strategy for each round and is committed to before the game begins, allowing the follower to observe it in advance. The optimal DSP results in what's called the Dynamic Bayesian Stackelberg Equilibrium (DSE). The leader's strategy in these games is about learning and adapting to the responses of the follower to maximize the leader's utility, rather than directly learning the type of the follower as in traditional learning scenarios.",
│   type='Knowledge/RAG',
│   sources=['CSE_1.pdf']
)

In [31]:
# Or access individual fields
print(f"Query: {response.content.query}")
print(f"Answer: {response.content.answer}")
print(f"Type: {response.content.type}")
print(f"Sources: {response.content.sources}")

Query: Learning in Dynamic Bayesian Stackelberg Games
Answer: In Dynamic Bayesian Stackelberg Games, the focus is on how the leader can optimize their utility over multiple rounds of the game rather than merely identifying the follower's type. The leader employs a Dynamic Bayesian Stackelberg Policy (DSP), which specifies the strategy at each round based on the follower's past responses. This policy outlines a leader strategy for each round and is committed to before the game begins, allowing the follower to observe it in advance. The optimal DSP results in what's called the Dynamic Bayesian Stackelberg Equilibrium (DSE). The leader's strategy in these games is about learning and adapting to the responses of the follower to maximize the leader's utility, rather than directly learning the type of the follower as in traditional learning scenarios.
Type: Knowledge/RAG
Sources: ['CSE_1.pdf']


# Multi-Lingual Agent

In [33]:
# Agent with structured_output using Pydantic model
agent = Agent(
    tools=[DuckDuckGoTools()],
    knowledge=knowledge_base,
    instructions=[
        "Understand user's query and translate into English to use the knowledge-base and web search"
        "Try to answer the query using the knowledge base (RAG) first.",
        "If the knowledge base does not contain the information or the user asks for a web search, use DuckDuckGo.",
        "Return your response following the provided structured format."
        "Answer in the same language as of user's query"
    ],
    response_model=AnswerOutput,  # <-- Pydantic model here
    show_tool_calls=True,
    markdown=False,
)

In [34]:
from agno.agent import RunResponse
from rich.pretty import pprint
from datetime import datetime

start_time = datetime.now()
print(f"Start Time: {start_time.strftime('%Y-%m-%d %H:%M:%S')}")

# Test the agent
# response: RunResponse = agent.run("What are the latest developments in AI, and how do they relate to the concepts in our knowledge base?")
response: RunResponse = agent.run("ডায়নামিক বেয়েসিয়ান স্ট্যাকেলবার্গ গেমসে শেখা সম্পর্কে আমাকে বলুন")

end_time = datetime.now()
elapsed = end_time - start_time

print(f"End Time: {end_time.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Elapsed Time: {elapsed.total_seconds():.2f} seconds")  # <-- FIXED


Start Time: 2025-06-08 15:15:18
End Time: 2025-06-08 15:15:29
Elapsed Time: 10.14 seconds


In [35]:
# Pretty print the structured content (your AnswerOutput model)
pprint(response.content)

AnswerOutput(
│   query='ডায়নামিক বেয়েসিয়ান স্ট্যাকেলবার্গ গেমসে শেখা সম্পর্কে আমাকে বলুন',
│   answer='ডায়নামিক বেয়েসিয়ান স্ট্যাকেলবার্গ গেমগুলিতে শেখা হলো একটি প্রক্রিয়া যেখানে এক বা একাধিক নেতা (লিডার) এবং অনুসারী (ফলোয়ার) তাদের নীতিগুলিকে উন্নত করতে শেখে। এই ধরনের গেমগুলিতে শেখা শুধুমাত্র যোগাযোগের বিকল্প নয় বরং এর মাধ্যমে একটি কার্যকর ডায়নামিক নীতি নিরূপণ করা সম্ভব। যখন সময় পরিসীমা (T) ১ হয় তখন ডায়নামিক স্ট্যাকেলবার্গ সমীকরণ (DSE) বেয়েসিয়ান স্ট্যাকেলবার্গ সমীকরণে (BSE) রুপান্তরিত হয়, যা NP-কঠিন সমস্যা হিসেবে পরিচিত। T বেশি হলে, ডায়নামিক নীতি সম্পূর্ণ করতে স্থান \x00 nT \x01 লাগতে পারে এবং এই সমস্যা সমাধানে সময় লাগে T এবং |Θ| এর উপর নির্ভরশীল। সমস্যার সমাধানে কার্যকর পদ্ধতি হিসেবে একটি মিশ্র সংখ্যা লিনিয়ার প্রোগ্রাম (MILP) প্রণয়ন করা হয়েছে, যা সময়ের সাথে T এবং |Θ| এর উপর নির্ভর করে কাজ করে।',
│   type='Knowledge/RAG',
│   sources=['CSE_1.pdf']
)

# Light RAG

In [37]:
# !pip install -q lightrag-hku
# !pip install -q pdfplumber
# !pip install nest_asyncio
# !pip install -q nest_asyncio

In [38]:
# Ensure your OpenAI key is set
import os
os.environ["OPENAI_API_KEY"] = openai_api_key

import nest_asyncio
nest_asyncio.apply()                                  

from pathlib import Path
from lightrag import LightRAG, QueryParam             # :contentReference[oaicite:0]{index=0}
from lightrag.llm.openai import gpt_4o_mini_complete, openai_embed
from lightrag.kg.shared_storage import initialize_pipeline_status
from lightrag.utils import setup_logger

setup_logger("lightrag", level="INFO")


2025-06-08 15:15:29 - pipmaster.package_manager - INFO - Targeting pip associated with Python: C:\Users\lenovo\anaconda3\python.exe | Command base: C:\Users\lenovo\anaconda3\python.exe -m pip


In [39]:
import shutil, os
WORKING_DIR = "./light_rag_storage"
if os.path.exists(WORKING_DIR):
    shutil.rmtree(WORKING_DIR)
os.makedirs(WORKING_DIR)

rag = LightRAG(
    working_dir=WORKING_DIR,
    embedding_func=openai_embed,
    llm_model_func=gpt_4o_mini_complete,
)

INFO: Process 3612 Shared-Data created for Single Process
INFO: Created new empty graph
INFO:nano-vectordb:Init {'embedding_dim': 1536, 'metric': 'cosine', 'storage_file': './light_rag_storage\\vdb_entities.json'} 0 data
INFO:nano-vectordb:Init {'embedding_dim': 1536, 'metric': 'cosine', 'storage_file': './light_rag_storage\\vdb_relationships.json'} 0 data
INFO:nano-vectordb:Init {'embedding_dim': 1536, 'metric': 'cosine', 'storage_file': './light_rag_storage\\vdb_chunks.json'} 0 data


In [40]:
await rag.initialize_storages()
await initialize_pipeline_status()


INFO: Process 3612 initialized updated flags for namespace: [full_docs]
INFO: Process 3612 ready to initialize storage namespace: [full_docs]
INFO: Process 3612 KV load full_docs with 0 records
INFO: Process 3612 initialized updated flags for namespace: [text_chunks]
INFO: Process 3612 ready to initialize storage namespace: [text_chunks]
INFO: Process 3612 KV load text_chunks with 0 records
INFO: Process 3612 initialized updated flags for namespace: [entities]
INFO: Process 3612 initialized updated flags for namespace: [relationships]
INFO: Process 3612 initialized updated flags for namespace: [chunks]
INFO: Process 3612 initialized updated flags for namespace: [chunk_entity_relation]
INFO: Process 3612 initialized updated flags for namespace: [llm_response_cache]
INFO: Process 3612 ready to initialize storage namespace: [llm_response_cache]
INFO: Process 3612 KV load llm_response_cache with 0 records
INFO: Process 3612 initialized updated flags for namespace: [doc_status]
INFO: Proces

In [41]:
from pathlib import Path                                  # need Path import
from langchain.document_loaders import PyPDFLoader, Docx2txtLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 1) Locate all source files
DATA_DIR = Path("./data")
raw_docs = []

for file_path in DATA_DIR.iterdir():
    suffix = file_path.suffix.lower()
    if suffix == ".pdf":
        loader = PyPDFLoader(str(file_path))
    elif suffix == ".docx":
        loader = Docx2txtLoader(str(file_path))
    elif suffix == ".txt":
        loader = TextLoader(str(file_path), encoding="utf-8")
    else:
        continue

    docs = loader.load()                                  # :contentReference[oaicite:0]{index=0}
    for doc in docs:
        doc.metadata["source"] = file_path.name           # track origin for citations
    raw_docs.extend(docs)

# 2) Split into ~1 000-token chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks   = splitter.split_documents(raw_docs)            # :contentReference[oaicite:1]{index=1}

# 3) Prepare for ingestion
texts      = [chunk.page_content       for chunk in chunks]
file_paths = [chunk.metadata["source"] for chunk in chunks]

print(f"Prepared {len(chunks)} chunks from {len(raw_docs)} documents")
print(file_paths)                                        # verify your sources


Prepared 179 chunks from 53 documents
['astro_physics_1.pdf', 'astro_physics_1.pdf', 'astro_physics_1.pdf', 'astro_physics_1.pdf', 'astro_physics_1.pdf', 'astro_physics_1.pdf', 'astro_physics_1.pdf', 'astro_physics_1.pdf', 'astro_physics_1.pdf', 'astro_physics_1.pdf', 'astro_physics_1.pdf', 'astro_physics_1.pdf', 'astro_physics_1.pdf', 'astro_physics_1.pdf', 'astro_physics_1.pdf', 'astro_physics_1.pdf', 'CSE_1.pdf', 'CSE_1.pdf', 'CSE_1.pdf', 'CSE_1.pdf', 'CSE_1.pdf', 'CSE_1.pdf', 'CSE_1.pdf', 'CSE_1.pdf', 'CSE_1.pdf', 'CSE_1.pdf', 'CSE_1.pdf', 'CSE_1.pdf', 'CSE_1.pdf', 'CSE_1.pdf', 'CSE_1.pdf', 'CSE_1.pdf', 'CSE_1.pdf', 'CSE_1.pdf', 'CSE_1.pdf', 'CSE_1.pdf', 'CSE_1.pdf', 'CSE_1.pdf', 'CSE_1.pdf', 'CSE_1.pdf', 'CSE_1.pdf', 'CSE_1.pdf', 'CSE_1.pdf', 'CSE_1.pdf', 'CSE_1.pdf', 'CSE_1.pdf', 'CSE_1.pdf', 'CSE_1.pdf', 'CSE_1.pdf', 'CSE_1.pdf', 'CSE_1.pdf', 'CSE_1.pdf', 'CSE_1.pdf', 'CSE_1.pdf', 'CSE_1.pdf', 'CSE_1.pdf', 'CSE_1.pdf', 'CSE_1.pdf', 'CSE_1.pdf', 'CSE_1.pdf', 'CSE_1.pdf', 'CSE_1.p

In [42]:
# Build unique chunk IDs to bypass dedupe
chunk_ids = [f"{src}_{i}" for i, src in enumerate(file_paths)]

# Synchronous (blocking) insert:
await rag.ainsert(
    texts,
    ids=chunk_ids,
    file_paths=file_paths,
    split_by_character_only=True
)

INFO: Stored 179 new unique documents
INFO: Processing 179 document(s)
INFO: Extracting stage 1/179: CSE_1.pdf
INFO: Processing d-id: CSE_1.pdf_96
INFO: Extracting stage 2/179: CSE_1.pdf
INFO: Processing d-id: CSE_1.pdf_85
INFO: limit_async: 16 new workers initialized
INFO: limit_async: 4 new workers initialized
INFO:  == LLM cache == saving default: 61f2011035702e91f33baae2b4311957
INFO:  == LLM cache == saving default: 80d4739eeca13f58f9d5fd52f5dff2ab
INFO: Chunk 1 of 1 extracted 3 Ent + 2 Rel
INFO: Merging stage 2/179: CSE_1.pdf
INFO: Updating 3 entities  2/179: CSE_1.pdf
INFO: Extracting stage 3/179: CSE_1.pdf
INFO: Processing d-id: CSE_1.pdf_45
INFO:  == LLM cache == saving default: ecfd80d73af58540115b54a3720e0727
INFO: Updating 2 relations 2/179: CSE_1.pdf
INFO: Writing graph with 3 nodes, 2 edges
INFO: In memory DB persist to disk
INFO: Completed processing file 2/179: CSE_1.pdf
INFO:  == LLM cache == saving default: 03dc6904cf098f7c30f2bcfb513af419
INFO:  == LLM cache == savin

In [43]:
query = "Tell me about Learning in Dynamic Bayesian Stackelberg Games"
print(
  rag.query(
    query,
    param=QueryParam(mode="hybrid")
  )
)



INFO:  == LLM cache == saving hybrid: 5364db14231309f88fb8bb595e80e0dc
INFO: Process 3612 building query context...
INFO: Query nodes: Bayesian strategies, Stackelberg competition, Player types, Information structure, Decision-making, top_k: 60, cosine: 0.2
INFO: Local query uses 54 entites, 141 relations, 17 chunks
INFO: Query edges: Learning, Dynamic Bayesian Stackelberg Games, Game theory, top_k: 60, cosine: 0.2
INFO: Global query uses 25 entites, 60 relations, 19 chunks
INFO:  == LLM cache == saving hybrid: 487b7c6a7b960db376a201d26d7d753d


### Learning in Dynamic Bayesian Stackelberg Games

Dynamic Bayesian Stackelberg Games are characterized by repeated interactions between a leader and a follower, where the follower's type is uncertain, modeled as a random variable known only to the follower. The leader, who commits to a strategy first, can only maintain a prior distribution on the follower's type (denoted as µ), which influences the leader's decision-making process.

**Effective Learning Process**

Effective learning in this context occurs when the leader can adapt their strategies based on the observed behaviors of the follower. A critical component of this learning process is Assumption 1, which ensures the existence of a learnable subgroup among the follower types, thereby facilitating effective strategy adaptation by the leader. This learning is driven by the distinct best responses exhibited by the follower types, allowing the leader to tailor their policies accordingly.

**Contrast to No Learning Theorem**

The 

In [44]:
await rag.finalize_storages()

# LightRAG with Agno

In [102]:
# rag_tool.py

from agno.tools import tool

# @tool(
#     name="rag_query_tool",
#     description="Answer arbitrary queries using RAG in hybrid mode",
#     show_result=True
# )
def rag_query_tool(query: str) -> str:
    """
    Takes a user’s query and returns the RAG-generated answer using hybrid search.
    """
    # 👇 Replace this import with the real location of your 'rag' and 'QueryParam'

    return rag.query(
        query,
        param=QueryParam(mode="hybrid")
    )


In [104]:
# Define Pydantic model for structured output
class AnswerOutput(BaseModel):
    query: str = Field(..., description="The user's original query.")
    answer: str = Field(..., description="The answer to the user's query.")
    type: str = Field(..., description="Source of the answer: 'Knowledge/RAG' or 'Web Search'.")
    sources: List[str] = Field(..., description="List of sources used (URLs or document names).")


In [120]:
from agno.agent import RunResponse
from rich.pretty import pprint
from datetime import datetime

start_time = datetime.now()
print(f"Start Time: {start_time.strftime('%Y-%m-%d %H:%M:%S')}")

# Test the agent
# response: RunResponse = agent.run("ডায়নামিক বেয়েসিয়ান স্ট্যাকেলবার্গ গেমসে শেখা সম্পর্কে আমাকে বলুন")
response: RunResponse = agent.run("Tell me about Learning in Dynamic Bayesian Stackelberg Games")

end_time = datetime.now()
elapsed = end_time - start_time

print(f"End Time: {end_time.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Elapsed Time: {elapsed.total_seconds():.2f} seconds")  # <-- FIXED


Start Time: 2025-06-08 16:57:58
End Time: 2025-06-08 16:58:06
Elapsed Time: 7.58 seconds


In [124]:
print(response.content)

query='Learning in Dynamic Bayesian Stackelberg Games' answer="Dynamic Bayesian Stackelberg Games (DBSGs) represent a strategic decision-making framework where a leader and follower interact under uncertainty regarding the follower’s type. The leader must make decisions based on incomplete information about the follower's payoff matrix, capturing the essence of strategic interactions where learning can be crucial for optimizing outcomes.\n\n### Effective Learning\n\nIn DBSGs, effective learning occurs when a leader can optimize their utility over multiple rounds based on the behavior of the follower. The primary goal for the leader is not merely identifying the follower's type but maximizing returns over interactions. This involves leveraging revealed preferences of the follower to distinguish types, allowing the leader to adapt and enhance strategies accordingly. The Dynamic Bayesian Stackelberg Equilibrium (DSE) encapsulates this optimal dynamic policy that combines learning with uti

In [126]:
from agno.agent import RunResponse
from rich.pretty import pprint
from datetime import datetime

start_time = datetime.now()
print(f"Start Time: {start_time.strftime('%Y-%m-%d %H:%M:%S')}")

# Test the agent
response: RunResponse = agent.run("ডায়নামিক বেয়েসিয়ান স্ট্যাকেলবার্গ গেমসে শেখা সম্পর্কে আমাকে বলুন")
# response: RunResponse = agent.run("Tell me about Learning in Dynamic Bayesian Stackelberg Games")

end_time = datetime.now()
elapsed = end_time - start_time

print(f"End Time: {end_time.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Elapsed Time: {elapsed.total_seconds():.2f} seconds")  # <-- FIXED


Start Time: 2025-06-08 16:59:17


INFO:  == LLM cache == saving hybrid: 9f1c20b915487f4819082e20206e8f82
INFO: Process 3612 building query context...
INFO: Query nodes: গেম থিওরি, অর্থনৈতিক সিদ্ধান্ত, নীতিনির্ধারণ, অন্তর্ভুক্তি, সামাজিক আচরণ, top_k: 60, cosine: 0.2
INFO: Local query uses 27 entites, 76 relations, 13 chunks
INFO: Query edges: ডায়নামিক বেয়েসিয়ান, স্ট্যাকেলবার্গ গেম, শেখা, top_k: 60, cosine: 0.2
INFO: Global query uses 29 entites, 60 relations, 19 chunks
INFO:  == LLM cache == saving hybrid: 0113e4b459c26d1aed37a84b5246c37e


End Time: 2025-06-08 16:59:52
Elapsed Time: 35.13 seconds


In [127]:
print(response.content)

query='ডায়নামিক বেয়েসিয়ান স্ট্যাকেলবার্গ গেমসে শেখা সম্পর্কে আমাকে বলুন' answer='ডায়নামিক বেয়েসিয়ান স্ট্যাকেলবার্গ গেমসে শেখার প্রক্রিয়া এমন একটি খেলার প্রেক্ষাপট তৈরি করে যেখানে নেতা এবং অনুসারী উভয়েরই অপর্যাপ্ত তথ্য থাকে। এই গেমে, নেতা তার কৌশলগুলি সেট করে যেমন, নেতৃত্ব দেওয়ার সময়, অনুসারী সেই কৌশলগুলির প্রতিক্রিয়া দেয়। শেখার এই প্রক্রিয়াটি বিশেষভাবে গুরুত্বপূর্ণ কারণ এটি নেতা তাদের কৌশলগুলোকে নিখুঁত করতে এবং তাদের উপকারিতা বাড়াতে সহায়তা করে।\n\n**শেখার কার্যকরী প্রক্রিয়া**: \nডায়নামিক বেয়েসিয়ান স্ট্যাকেলবার্গ গেমে শেখা কার্যকরীভাবে ঘটে যখন নেতা কথোপকথনের পরিবর্তে তথ্য অধিগ্রহণের মাধ্যমে প্রত্যাশিত ফলাফলগুলিকে সংশোধন করে। গবেষণা দেখায় যে নেতা কৌশলগত প্রতিপক্ষের বিরুদ্ধে পুনরাবৃত্ত খেলার মধ্যে মাধ্যমে তার লাভ বাড়ানোর জন্য তথ্য অর্জন করতে পারে, এবং এটি শুধুমাত্র যোগাযোগের বিকল্প হিসাবে কাজ করে না। এই শেখার মাধ্যমে কিভাবে অনুসারী প্রকারভেদগুলি সনাক্ত করা যায় এবং সেগুলির সাথে কিভাবে প্রতিক্রিয়াশীল হতে হবে, তা শেখা যায়।\n\n**মিশ্র সংখ্যা লিনিয়ার প্রোগ্রাম (MILP)**